In [23]:
import requests

### Проверим доступность сервиса

In [24]:
response = requests.get('http://127.0.0.1:5000/hello')
response.content

b'{"response":{"blah":["stuff"]}}\n'

## Доступна ли ветка обучения

In [25]:
response = requests.get('http://127.0.0.1:5000/train')
response.content

b'{"response":{"status":"work"}}\n'

## Отправим модель для обучения

In [26]:
ret = requests.post('http://127.0.0.1:5000/train', data={
    "data": "a,c,b\n1.0,1.0,2.0\n4.0,4.0,2.0\n8.0,7.0,2.0\n11.0,12.0,2.0",
    "target": "a",  # предсказываемая переменная
    "n_folds": "2",  # число кусков для k-fold
    "fit_intercept": "True",  # включать в модель свободный член
    "l2_coef": "[0.01, 0.1, 1.0, 10.0, 100.0]"
    })

#### посмотрим её id и найденные параметры параметры (БД проверяется на новые модели каждые 10 сек. поэтому необходимо подождать)

In [27]:
model_id = eval(ret.content)['response']['model_id']
model_id

11

In [28]:
ret = requests.get(
    'http://127.0.0.1:5000/model/{}'.format(model_id)
)
ret.content

b'{"response":{"cv_results":{"0.01":{"mean_mse":4.604430607065071},"0.1":{"mean_mse":4.6569075802974025},"1.0":{"mean_mse":6.020406081012142},"10.0":{"mean_mse":23.303871052979257},"100.0":{"mean_mse":45.72691762754719}},"model":{"coef":{"b":0.0,"c":0.9241024087259506},"intercept":0.45538554764429584}}}\n'

## Отправим запрос на предсказание данной модели

In [29]:
ret = requests.post(
    'http://127.0.0.1:5000/model/{}/predict'.format(model_id),
    data={
        "data": "c,b\n2.0,2.0\n10.0,2.0",
        "model_id": model_id
    }
)

ret.content

b'{"response":{"result":"[2.30359037 9.69640963]"}}\n'

In [30]:
ret.status_code

200

# Обработка ошибок

#### NaN в тренировочных данных

In [31]:
ret = requests.post('http://127.0.0.1:5000/train', data={
    "data": "a,c,b\n1.0,1.0,2.0\n4.0,4.0,2.0\n8.0,7.0,2.0\n11.0,12.0",
    "target": "a",  # предсказываемая переменная
    "n_folds": "2",  # число кусков для k-fold
    "fit_intercept": "True",  # включать в модель свободный член
    "l2_coef": "[0.01, 0.1, 1.0, 10.0, 100.0]"
    })
ret.content

b'{"response":{"error":"\'data\' contain NaN"}}\n'

#### колонка которую надо предсказать не существует

In [32]:
ret = requests.post('http://127.0.0.1:5000/train', data={
    "data": "a,c,b\n1.0,1.0,2.0\n4.0,4.0,2.0\n8.0,7.0,2.0\n11.0,12.0,2.0",
    "target": "k",  # предсказываемая переменная
    "n_folds": "2",  # число кусков для k-fold
    "fit_intercept": "True",  # включать в модель свободный член
    "l2_coef": "[0.01, 0.1, 1.0, 10.0, 100.0]"
    })
ret.content

b'{"response":{"error":"\'target\' column noi in data"}}\n'

#### вызываем модель, но она ещё не обучена

In [33]:
ret_id = requests.post('http://127.0.0.1:5000/train', data={
    "data": "a,c,b\n1.0,1.0,2.0\n4.0,4.0,2.0\n8.0,7.0,2.0\n11.0,12.0,2.0",
    "target": "a",  # предсказываемая переменная
    "n_folds": "2",  # число кусков для k-fold
    "fit_intercept": "True",  # включать в модель свободный член
    "l2_coef": "[0.01, 0.1, 1.0, 10.0, 100.0]"
    })
eval(ret_id.content)['response']['model_id']

12

In [34]:
mod_id = eval(ret_id.content)['response']['model_id']

ret = requests.post(
    'http://127.0.0.1:5000/model/{}/predict'.format(mod_id),
    data={
        "data": "c,b\n2.0,2.0\n10.0,2.0",
        "model_id": mod_id
    }
)
ret.content

b'{"response":{"error":"model in line for training"}}\n'

#### модель не существует

In [35]:
ret = requests.get(
    'http://127.0.0.1:5000/model/{}'.format("10000"),
    data={
        "data": "c,b\n2.0,2.0\n10.0,2.0",
        "model_id": 10000
    }
)
ret.content

b'{"response":{"error":"Not valid model_id"}}\n'

#### столбцы данных для обучения и предсказания не совпадают

In [36]:
ret = requests.post(
    'http://127.0.0.1:5000/model/{}/predict'.format(4),
    data={
        "data": "c,g\n2.0,2.0\n10.0,2.0",
        "model_id": 1
    }
)
ret.content

b'{"response":{"error":"data for prediction is not valid for this model"}}\n'

## Также есть и другие обработанные ошибки, которые связаны с валидацией данных, их доступности, корректности при чтении (read_csv) и т. п.